In [2]:
import sys
import os
import shutil
import time

import traceback
import signal as sg

from pathlib import Path
import h5py
import json
import pickle

import scipy
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)


import seaborn as sns
sns.set(style='whitegrid', palette='muted')

## added TreeMazeanalyses folder using the following command
## conda develop /home/alexgonzalez/Documents/TreeMazeAnalyses2
import TreeMazeAnalyses2.Utils.robust_stats as rs
import TreeMazeAnalyses2.Pre_Processing.pre_process_functions as pp
import TreeMazeAnalyses2.Sorting.sort_functions as sf

import spikeextractors as se
import spikesorters as ss
import spiketoolkit as st

sns.set(style= 'whitegrid', palette= 'muted')

import signal as sg

from importlib import reload

## get sorted sessions

In [21]:
subject_id = 'Li'
sorter = 'KS2'
nTT = 16

for subject_id in ['Li','Ne','Cl']:
    if subject_id in ['Cl']:
        root_path = Path('/Data2_SSD2T')
    else:
        root_path = Path('/Data_SSD2T')
    
    sort_path = Path(root_path, 'Data/Sorted', subject_id)    
    pp_path = Path(root_path, 'Data/PreProcessed', subject_id)

    task_table_file = pp_path / 'TasksDir' / ('sort_{}_{}.json'.format(subject_id,sorter))
    with task_table_file.open(mode='r') as f:
        task_table = json.load(f)

    sort_tt_table = pd.DataFrame(columns=np.arange(1,nTT+1))
    sort_tt_valid_table = pd.DataFrame(columns=np.arange(1,nTT+1))
    sort_tt_curated_table = pd.DataFrame(columns=np.arange(1,nTT+1))

    sort_summary_table = pd.DataFrame(index=list(task_table.keys()),columns=['session','n_tt','n_tt_sorted','n_tt_curated','n_total_valid_clusters' ,'n_total_cells','n_total_muas', 'n_total_noise','n_total_unsorted'])

    for task_id, task in task_table.items():
        session_name = task['session_name']
        n_tt = 0
        n_sorted = 0
        n_valid_clusters = 0
        n_curated = 0
        n_cells = 0 
        n_muas = 0
        n_noise = 0
        n_unsorted = 0 
        for subtask_id, subtask in task['files'].items():
            n_tt+=1
            tt_id = subtask['tt_id']

            cl_stat_file = Path(subtask['save_path'],sorter,'cluster_stats.csv')
            if cl_stat_file.exists():
                tt_sorted = 1.0
                tt_valid_clusters = pd.read_csv(cl_stat_file, index_col=0)['valid'].sum()
            else:
                tt_sorted = np.nan
                tt_valid_clusters = 0

            sort_tt_table.at[session_name,int(tt_id)] = tt_sorted
            sort_tt_valid_table.at[session_name,int(tt_id)] = tt_valid_clusters
            n_sorted = np.nansum((n_sorted,tt_sorted))
            n_valid_clusters+=tt_valid_clusters

            cl_group_file = Path(subtask['save_path'],sorter, 'cluster_group.tsv')
            if cl_group_file.exists():
                group_counts = pd.read_csv(cl_group_file,sep='\t')['group'].value_counts()

                group_types = list(group_counts.keys())
                if any([gt in group_types for gt in ['mua','good','noise']]):
                    tt_curated = 1.0 
                    dat_file = Path(subtask['save_path'],sorter,'recording.dat')
                    if dat_file.exists():
                       os.remove(str(dat_file))
                else:
                    tt_curated = 0.0
                if 'mua' in group_types:
                    n_muas += group_counts.at['mua']
                if 'noise' in group_types:
                    n_noise +=  group_counts.at['noise'] 
                if 'good' in group_types:
                    n_cells += group_counts.at['good']
                if 'unsorted' in group_types:
                    n_unsorted += group_counts.at['unsorted']
            else:
                tt_curated = 0.0

            sort_tt_curated_table.at[session_name,int(tt_id)] = tt_curated
            n_curated += tt_curated

        sort_summary_table.at[task_id,'session'] = session_name
        sort_summary_table.at[task_id,'n_tt'] = n_tt
        sort_summary_table.at[task_id,'n_tt_sorted'] = n_sorted
        sort_summary_table.at[task_id,'n_tt_curated'] = n_curated
        sort_summary_table.at[task_id,'n_total_valid_clusters'] = n_valid_clusters
        sort_summary_table.at[task_id,'n_total_cells'] = n_cells
        sort_summary_table.at[task_id,'n_total_muas'] = n_muas
        sort_summary_table.at[task_id,'n_total_noise'] = n_noise
        sort_summary_table.at[task_id,'n_total_unsorted'] = n_unsorted
        
        sort_summary_table.to_csv(Path(sort_path,('sort_summary_{}_{}.csv'.format(sorter,subject_id))))      
        sort_tt_table.to_csv(Path(sort_path,('sort_tt_{}_{}.csv'.format(sorter,subject_id))))      
        sort_tt_valid_table.to_csv(Path(sort_path,('sort_valid_{}_{}.csv'.format(sorter,subject_id))))      
        sort_tt_curated_table.to_csv(Path(sort_path,('sort_curated_{}_{}.csv'.format(sorter,subject_id))))      
    

In [26]:
pd.read_csv(cl_group_file,sep='\t')['group'].unique()

array(['unsorted'], dtype=object)

In [28]:
a,b,c=1,1,1

In [14]:
sort_summary_table.to_csv(Path(sort_path,('sort_summary_{}_{}.csv'.format(sorter,subject_id))))
sort_summary_table

,session,n_tt,n_tt_sorted,n_tt_curated,n_total_valid_clusters,n_total_cells,n_total_muas,n_total_noise,n_total_unsorted
1,Li_T3g_052818,3,3,0,2,0,0,0,50
2,Li_OF_052818,8,7,7,42,3,11,110,18
3,Li_T3g_052918,13,10,0,43,0,0,0,153
4,Li_OF_052918,12,10,10,77,16,13,136,0
5,Li_OF_053018,13,11,10,56,12,28,125,17
6,Li_T3g_053018,13,11,0,40,0,0,0,156
7,Li_OF_053118,11,9,8,85,8,15,115,4
8,Li_T3g_053118,14,12,0,57,0,0,0,184
9,Li_T3g_060118,14,11,0,39,0,0,0,161
10,Li_OF_060118,14,10,9,54,8,13,133,7


True

In [19]:
sort_tt_table.to_csv(Path( data_folder ,('valid_session_tt_{}.csv'.format(subject_id))))
sort_tt_valid_table.to_csv(Path( data_folder ,('valid_tt_sorts_{}.csv'.format(subject_id))))


In [5]:
(1-(sort_id_table['n_tt']-sort_id_table['n_sorted'])/sort_id_table['n_tt']).median()

0.7857142857142857

In [70]:
np.nansum(sort_table.values.astype(float),axis=1)

array([ 3.,  8., 10., 10., 11., 11.,  9., 12., 11., 10.,  8.,  9., 11.,
       11., 11.,  6., 12., 14.,  8.,  7.,  9.,  0., 10., 10., 10., 13.,
        9.,  7.,  9., 10.,  9., 11.,  9., 11.,  8., 11., 10., 11., 10.,
        9.,  9.,  9., 11., 10.,  9.,  0.,  9., 11., 11., 10., 10., 11.,
       11.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [71]:
sort_table

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Li_T3g_052818,NaN,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Li_OF_052818,NaN,1,NaN,1,1,1,1,1,NaN,NaN,1,NaN,1,NaN,NaN,NaN
Li_T3g_052918,0,1,1,0,NaN,0,1,1,1,1,NaN,NaN,1,1,1,1
Li_OF_052918,0,1,1,0,NaN,1,1,1,1,1,NaN,NaN,1,1,NaN,1
Li_OF_053018,1,1,1,0,0,1,1,1,1,1,NaN,NaN,1,1,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Li_T3g_080218,0,0,0,0,NaN,0,0,0,0,0,NaN,NaN,0,0,0,0
Li_OF_080218,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN
Li_T3g_080318,0,0,0,0,NaN,0,0,0,0,0,NaN,NaN,0,0,0,0
Li_T3g_080618,0,0,0,0,NaN,0,0,0,0,0,NaN,NaN,NaN,0,0,NaN


## convert sessions to np.float16

In [36]:
# task table folder
subject = 'Li'
data_root = '/home/alexgonzalez/Documents/TreeMazeAnalyses2/Pre_Processing/TasksDir/'
file_name = Path(data_root, 'pp_table_{}.json'.format(subject))

with file_name.open(mode="r") as f:
    task_table = json.load(f)

In [39]:
task_table['4']['Files']['3']

{'session': 'Li_OF_052818',
 'task_type': 'tt',
 'filenames': ['/Data_SSD2T/Data/Raw/Li/Li_OF_052818/CSC1a.ncs',
  '/Data_SSD2T/Data/Raw/Li/Li_OF_052818/CSC1b.ncs',
  '/Data_SSD2T/Data/Raw/Li/Li_OF_052818/CSC1c.ncs',
  '/Data_SSD2T/Data/Raw/Li/Li_OF_052818/CSC1d.ncs'],
 'tt_id': '1',
 'sp': '/Data_SSD2T/Data/PreProcessed/Li/Li_OF_052818',
 'subSessionID': '0000'}

In [46]:
for task_id, task in task_table.items():
    for subtask_id, subtask in task['Files'].items():
        if subtask['task_type']=='tt':
            tt_id = subtask['tt_id']
            data_file = Path( subtask['sp'], ('tt_{}.npy').format(tt_id) )
            signals = np.load(str(data_file))
            signals = signals.astype(np.float16)
            np.save(str(data_file),signals)

In [45]:
signals

array([[-5.4102e+00, -1.6670e+00,  1.7700e-01, ...,  1.2656e+01,
         1.0781e+01,  8.6562e+00],
       [ 3.8550e+02,  3.9025e+02,  3.9425e+02, ...,  4.2148e+00,
         4.0859e+00,  3.9609e+00],
       [-9.2266e+00, -5.7500e+00, -3.6680e+00, ...,  1.2861e+00,
        -5.2686e-01, -2.4941e+00],
       [-1.3938e+01, -1.0641e+01, -9.9844e+00, ...,  1.3586e+01,
         1.0203e+01,  4.4648e+00]], dtype=float16)

## Debug preprocessing

In [60]:
subject_id = 'Ne'
data_folder = Path('/Data_SSD2T/Data/PreProcessed/', subject_id)
task_id = 14

task_table_file = data_folder / 'TasksDir' / ('pp_table_{}.json'.format(subject_id))
with task_table_file.open(mode='r') as f:
    task_table = json.load(f)

subtask = task_table[str(task_id)]['files']

In [61]:
subtask_id = 13

pp.process_tetrode(subtask[str(subtask_id)])

In [62]:
tt_info = pp.get_tt_info(subtask[str(subtask_id)])

job = subtask[str(subtask_id)]

In [63]:
job

{'session': 'NE_OF_060418',
 'task_type': 'tt',
 'filenames': ['/media/alexgonzalez/RawData/Data/Ne/NE_OF_060418/CSC11a.ncs',
  '/media/alexgonzalez/RawData/Data/Ne/NE_OF_060418/CSC11b.ncs',
  '/media/alexgonzalez/RawData/Data/Ne/NE_OF_060418/CSC11c.ncs',
  '/media/alexgonzalez/RawData/Data/Ne/NE_OF_060418/CSC11d.ncs'],
 'tt_id': '11',
 'sp': '/Data_SSD2T/Data/PreProcessed/Ne/NE_OF_060418',
 'subSessionID': '0000'}